In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from string import punctuation

# Load dữ liệu

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/XLNNTN/TH/05/uit-visfd-clf.csv', sep='\t')
df.head()

,text,label,split
0,em học ở phòng mà bên cạnh là công_trình đang ...,0,train
1,máy_chiếu trong phòng học quá mờ khiến sinh_vi...,0,train
2,mong nhà_trường nâng_cấp các phòng thực_hành v...,0,train
3,thiếu kinh_nghiệm truyền_đạt .,0,train
4,nên dạy thực_tế hơn một_chút và máy_chiếu nên ...,0,train


# Khảo sát và tiền xử lý dữ liệu

In [ ]:
def vi_clean_str(input_str: str) -> str:
    my_punctuation = punctuation.replace('_', '')
    output_str = input_str.translate(str.maketrans('', '', my_punctuation))
    output_str = output_str.replace('_,', '')
    output_str = output_str.replace('_.', '')
    output_str = output_str.replace('…', '')
    output_str = output_str.replace('-', '_')
    output_str = output_str.replace('–', '_')
    output_str = output_str.replace('\u200b_\u200b', '')
    output_str = output_str.replace('\u200b', '')
    output_str = output_str.replace('‘', '_')
    output_str = output_str.replace('’', '_')
    output_str = output_str.replace('“', '_')
    output_str = output_str.replace('”', '_')
    output_str = ' '.join(output_str.split())
    return output_str.lower()


df['clean_text'] = df['text'].apply(vi_clean_str)
df.head()

,text,label,split,clean_text
0,em học ở phòng mà bên cạnh là công_trình đang ...,0,train,em học ở phòng mà bên cạnh là công_trình đang ...
1,máy_chiếu trong phòng học quá mờ khiến sinh_vi...,0,train,máy_chiếu trong phòng học quá mờ khiến sinh_vi...
2,mong nhà_trường nâng_cấp các phòng thực_hành v...,0,train,mong nhà_trường nâng_cấp các phòng thực_hành v...
3,thiếu kinh_nghiệm truyền_đạt .,0,train,thiếu kinh_nghiệm truyền_đạt
4,nên dạy thực_tế hơn một_chút và máy_chiếu nên ...,0,train,nên dạy thực_tế hơn một_chút và máy_chiếu nên ...


In [ ]:
corpus = df['clean_text']

min_len = 1_000_000
max_len = 0
total_len = 0

vocab = set()

for doc in corpus:
  words = doc.split()

  doc_len = len(words)
  total_len += doc_len

  if doc_len < min_len:
    min_len = doc_len

  if doc_len > max_len:
    max_len = doc_len

  for w in words:
    vocab.add(w)

corpus_size = len(corpus)
vocab = list(vocab)
vocab_size = len(vocab)

# statistic
print(f"Docs: {corpus_size}")
print(f"Vocab size: {vocab_size}")
print(f"Min length: {min_len}")
print(f"Max length: {max_len}")
print(f"Avg length: {total_len / corpus_size}")

Docs: 16175
Vocab size: 4214
Min length: 1
Max length: 123
Avg length: 9.49564142194745


In [ ]:
train_df = df[df['split'] == 'train']
val_df = df[df['split'] == 'val']
test_df = df[df['split'] == 'test']
print('train')
print(train_df[:5])
print('validation')
print(val_df[:5])
print('test')
print(test_df[:5])

print(f"Train size: {len(train_df)}")
print(f"Val size: {len(val_df)}")
print(f"Test size: {len(test_df)}")

train
                                                text  label  split  \
0  em học ở phòng mà bên cạnh là công_trình đang ...      0  train   
1  máy_chiếu trong phòng học quá mờ khiến sinh_vi...      0  train   
2  mong nhà_trường nâng_cấp các phòng thực_hành v...      0  train   
3                     thiếu kinh_nghiệm truyền_đạt .      0  train   
4  nên dạy thực_tế hơn một_chút và máy_chiếu nên ...      0  train   

                                          clean_text  
0  em học ở phòng mà bên cạnh là công_trình đang ...  
1  máy_chiếu trong phòng học quá mờ khiến sinh_vi...  
2  mong nhà_trường nâng_cấp các phòng thực_hành v...  
3                       thiếu kinh_nghiệm truyền_đạt  
4  nên dạy thực_tế hơn một_chút và máy_chiếu nên ...  
validation
                                                    text  label split  \
14592                 mong nhà_trường sớm thay máy mới !      0   val   
14593               cần nâng_cấp phòng học , máy_chiếu .      0   val   
14594  phòng 

In [ ]:
# Chuẩn bị dữ liệu
# Merge validation set và test set
train_text = train_df['clean_text'].tolist()
train_label = train_df['label'].tolist()

val_text = val_df['clean_text'].tolist()
val_label = val_df['label'].tolist()

"""
    Các mô hình học máy chưa cần đến tập validation trong quá trình huấn luyện
    Tạm thời gộp tập validation và tập test => tập test
"""
test_text = test_df['clean_text'].tolist()
test_label = test_df['label'].tolist()

test_text += val_text
test_label += val_label


print(len(train_text))
print(len(train_label))

print(len(test_text))
print(len(test_label))


print(train_text[:5])
print(test_text[:5])

11426
11426
4749
4749
['em học ở phòng mà bên cạnh là công_trình đang thi_công khá là ồn_ào vì_thế khi học đến phần nghe em không_thể nghe được gì và bài nghe bị gián_đoạn', 'máy_chiếu trong phòng học quá mờ khiến sinh_viên không nhìn rõ nội_dung khi trình_chiếu', 'mong nhà_trường nâng_cấp các phòng thực_hành và các thiết_bị trong phòng học', 'thiếu kinh_nghiệm truyền_đạt', 'nên dạy thực_tế hơn một_chút và máy_chiếu nên rõ hơn']
['cơ_sở vật_chất còn ảnh_hưởng đến chất_lượng môn_học', 'vì đặc_thù môn mạng máy_tính lại cần có mạng', 'chỉ có cơ_sở vật_chất trường còn hạn_chế', 'hy_vọng nhà_trường nâng_cấp các máy cũ trong phòng máy thực_hành để sinh_viên bọn em có đủ số máy để học_tập', 'cần thay_đổi hệ_thống máy_tính ở một_số phòng thực_hành vì đã quá cũ']


In [ ]:
from random import shuffle


"""
    Do khi merge dữ liệu từ bản nguyên mẫu, tool scan từng split, category -> dữ liệu
    có sữ sắp xếp thứ tự. Cần trộn lại tránh 1 số hiệu ứng không đáng có trong quá trình
    huấn luyện.
"""

def shuffle_data(data: list, labels: list):
    if (len(data) != len(labels)):
        raise ValueError('data and labels must be equal!')

    indices = [i for i in range(len(data))]
    shuffle(indices)

    new_data = []
    new_labels = []

    for idx in indices:
        new_data.append(data[idx])
        new_labels.append(labels[idx])

    return new_data, new_labels


train_text, train_label = shuffle_data(train_text, train_label)
test_text, test_label = shuffle_data(test_text, test_label)

# Print thử một vài dòng, chắc chắn quá trình shuffle an toàn

for i in range(10):
    print(f"{train_text[i]} --- {train_label[i]}")

bài_tập ít đa_dạng --- 2
giáo_viên trẻ_trung năng_động nhiệt_tình vui_vẻ --- 1
ngoài_ra giảng_viên còn tư_vấn định_hướng phát_triển nghề_nghiệp cho sinh_viên trong tương_lai --- 1
trang_bị thêm sách bài_tập --- 0
tạo hoạt_động thực_hành theo nhóm rất hay --- 1
giảng_viên dạy nhiệt_tình dễ hiểu --- 1
giáo_viên tận_tình giảng_dạy --- 1
giảng bài cảm_thấy hơi buồn_ngủ --- 1
chưa hiểu bài lắm --- 1
chất_lượng phòng máy rất tệ --- 0


# Huấn luyện và dự đoán

In [ ]:
# CountVectorizer = BoW
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(corpus)

X_train = vectorizer.transform(train_text)
y_train = train_label

X_test = vectorizer.transform(test_text)
y_test = test_label

In [ ]:
# Phân rã 1 vector để hiểu rõ hơn về bản chất của CountVectorizer
sentence = "cho ít bài_tập ví_dụ cho dễ bài_tập cho khó hơn nhiều thầy chép lý_thuyết trên bảng và đọc lại y_chang sinh_viên không có kỹ_năng giải bài_tập một_cách uyển_chuyển"
input_data = [sentence]

X = vectorizer.transform(input_data)

In [ ]:
x = X[0]

print(type(x))
print(x.shape)
print(x)

<class 'scipy.sparse._csr.csr_matrix'>
(1, 4187)
  (0, 168)	3
  (0, 217)	1
  (0, 337)	3
  (0, 385)	1
  (0, 585)	1
  (0, 839)	1
  (0, 970)	1
  (0, 1190)	1
  (0, 1373)	1
  (0, 1384)	1
  (0, 1493)	1
  (0, 1591)	1
  (0, 1618)	1
  (0, 1819)	1
  (0, 1973)	1
  (0, 2480)	1
  (0, 2789)	1
  (0, 2995)	1
  (0, 3298)	1
  (0, 3338)	1
  (0, 3351)	1
  (0, 3842)	1
  (0, 3875)	1
  (0, 4091)	1


In [ ]:
for word in sentence.split():
    print(f"{word}: {vectorizer.vocabulary_[word]}")

# In từ vựng và thứ tự của nó trong vectorizer, dò với kết quả ở trên.

cho: 337
ít: 3875
bài_tập: 168
ví_dụ: 3351
cho: 337
dễ: 839
bài_tập: 168
cho: 337
khó: 1373
hơn: 1190
nhiều: 1973
thầy: 2789
chép: 385
lý_thuyết: 1591
trên: 2995
bảng: 217
và: 3338
đọc: 4091
lại: 1618
y_chang: 3842
sinh_viên: 2480
không: 1384
có: 585
kỹ_năng: 1493
giải: 970
bài_tập: 168
một_cách: 1819
uyển_chuyển: 3298


In [ ]:
# Sau khi có feature vector, thì chúng ta sẽ chọn 1 mô hình nào đó để phân loại
from sklearn.svm import SVC


classifier = SVC()
classifier.fit(X_train, y_train)

SVC()

In [ ]:
from sklearn.metrics import classification_report

y_pred = classifier.predict(X_test)
print(classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.9367    0.6884    0.7936       215
           1     0.8731    0.9558    0.9126      3441
           2     0.7044    0.6448    0.6733       839
           3     0.6964    0.1535    0.2516       254

    accuracy                         0.8459      4749
   macro avg     0.8027    0.6106    0.6578      4749
weighted avg     0.8367    0.8459    0.8296      4749



In [ ]:
# Dùng thêm gridsearch để vét cạn

from sklearn.model_selection import GridSearchCV

parameters = {'kernel':('poly', 'rbf', 'sigmoid'), 'C':[1, 3]}
svc = SVC()


clf = GridSearchCV(svc, parameters)
clf.fit(X_train, y_train)


print(sorted(clf.cv_results_.keys()))

['mean_fit_time', 'mean_score_time', 'mean_test_score', 'param_C', 'param_kernel', 'params', 'rank_test_score', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'std_fit_time', 'std_score_time', 'std_test_score']


In [ ]:
clf.cv_results_

{'mean_fit_time': array([8.82173014, 6.7900815 , 3.28662577, 8.64977713, 7.37743607,
        2.72583141]),
 'std_fit_time': array([0.34064605, 0.46878732, 0.38034028, 0.42718901, 0.37718621,
        0.34924673]),
 'mean_score_time': array([1.17387114, 1.13689218, 0.66915674, 1.09865355, 1.26198788,
        0.58380771]),
 'std_score_time': array([0.08689293, 0.0157695 , 0.05769005, 0.0256124 , 0.18083119,
        0.02557757]),
 'param_C': masked_array(data=[1, 1, 1, 3, 3, 3],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['poly', 'rbf', 'sigmoid', 'poly', 'rbf', 'sigmoid'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'poly'},
  {'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'sigmoid'},
  {'C': 3, 'kernel': 'poly'},
  {'C': 3, 'kernel': 'rbf'},
  {'C': 3, 'kernel': 'sigmoid'}],
 'spli

In [ ]:
#Sau khi có feature vector, thì chúng ta sẽ chọn 1 mô hình nào đó để phân loại
from sklearn.svm import SVC


classifier = SVC(C= 3, kernel= 'rbf')
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.9512    0.7256    0.8232       215
           1     0.8892    0.9468    0.9171      3441
           2     0.7030    0.6603    0.6810       839
           3     0.6015    0.3150    0.4134       254

    accuracy                         0.8524      4749
   macro avg     0.7862    0.6619    0.7087      4749
weighted avg     0.8437    0.8524    0.8442      4749

